In [1]:
#dependencies
from pprint import pprint
import pandas as pd
import numpy as np
import time
import timeit
import requests
import json
import re
from config import gkey

# start program run timer
start = timeit.default_timer()

# setting variables
arena_ltlg = []
x = 0

# NHL

In [2]:
# pulling the table and columns wanted
NHLurl = 'https://en.wikipedia.org/wiki/List_of_National_Hockey_League_arenas'
tables = pd.read_html(NHLurl, header=[0])
NHLdf = tables[0]
NHLteam_arena = NHLdf[['Team','Arena']]

# column cleanup
NHLteam_arena['ArenaAPI'] = [re.sub(r'\s+', '%20', x) for x in NHLteam_arena['Arena']]
NHLteam_arena['ArenaAPI'] = NHLteam_arena['ArenaAPI'].replace({'&': ''}, regex=True)

# for loop to build data structure
for arena in NHLteam_arena['ArenaAPI']:
    # found some location accuracy issues so fixed them 
    # Fixing Pittsburgh Penguins
    if arena == 'PPG%20Paints%20Arena':
        arena = 'PPG%20Arena'
    # Fixing Edmonton Oilers
    if arena == 'Rogers%20Place':
        arena = 'Rogers%20Place%20Arena'
    target_arena = arena

    # Build the endpoint URL
    target_url = (f'https://maps.googleapis.com/maps/api/geocode/json?'
        'address={0}&key={1}').format(target_arena, gkey)
    geo_data = requests.get(target_url).json()
    lat = geo_data['results'][0]['geometry']['location']['lat']
    lng = geo_data['results'][0]['geometry']['location']['lng']
    address = geo_data['results'][0]['formatted_address']
    arena_ltlg.append({"team" : NHLteam_arena['Team'][x],
                     "location" : [lat, lng],
                     "address" : address,
                     "arena" : NHLteam_arena['Arena'][x],
                      "sport" : "NHL"})
    x+=1
    
# pprint(arena_ltlg)

<ipython-input-2-930369937d18>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  NHLteam_arena['ArenaAPI'] = [re.sub(r'\s+', '%20', x) for x in NHLteam_arena['Arena']]
<ipython-input-2-930369937d18>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  NHLteam_arena['ArenaAPI'] = NHLteam_arena['ArenaAPI'].replace({'&': ''}, regex=True)


# NFL

In [3]:
NFLurl = 'https://en.wikipedia.org/wiki/List_of_current_National_Football_League_stadiums'
tables = pd.read_html(NFLurl, header=[0])
NFLdf = tables[0]
NFLteam_arena = NFLdf[['Team(s)','Name']]

NFLteam_arena['ArenaAPI'] = [re.sub(r'\s+', '%20', x) for x in NFLteam_arena['Name']]
NFLteam_arena['ArenaAPI'] = NFLteam_arena['ArenaAPI'].replace({'&': ''}, regex=True)

x = 0
for arena in NFLteam_arena['ArenaAPI'] :
    target_arena = arena

    target_url = (f'https://maps.googleapis.com/maps/api/geocode/json?'
        'address={0}&key={1}').format(target_arena, gkey)
    geo_data = requests.get(target_url).json()
    if target_arena != 'Highmark%20Stadium':
        lat = geo_data['results'][0]['geometry']['location']['lat']
        lng = geo_data['results'][0]['geometry']['location']['lng']
        address = geo_data['results'][0]['formatted_address']
    # Fixing Buffalo Bills
    else:
        lat = geo_data['results'][1]['geometry']['location']['lat']
        lng = geo_data['results'][1]['geometry']['location']['lng']
        address = geo_data['results'][1]['formatted_address']
    arena_ltlg.append({"team" : NFLteam_arena['Team(s)'][x],
                     "location" : [lat, lng],
                       "address" : address,
                     "arena" : NFLteam_arena['Name'][x],
                      "sport" : "NFL"})
    x+=1

# pprint(arena_ltlg)

<ipython-input-3-d6852a1c4046>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  NFLteam_arena['ArenaAPI'] = [re.sub(r'\s+', '%20', x) for x in NFLteam_arena['Name']]
<ipython-input-3-d6852a1c4046>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  NFLteam_arena['ArenaAPI'] = NFLteam_arena['ArenaAPI'].replace({'&': ''}, regex=True)


# MLB

In [4]:
MLBurl = 'https://en.wikipedia.org/wiki/List_of_current_Major_League_Baseball_stadiums'
tables = pd.read_html(MLBurl, header=[0])
MLBdf = tables[1]
MLBteam_arena = MLBdf[['Team','Name']]

MLBteam_arena['ArenaAPI'] = [re.sub(r'\s+', '%20', x) for x in MLBteam_arena['Name']]
MLBteam_arena['Team'] = MLBteam_arena['Team'].replace({'nb 2': '', 'nb 3':''}, regex=True)
MLBteam_arena['Team'] = MLBteam_arena['Team'].str.replace('[', '')
MLBteam_arena['Team'] = MLBteam_arena['Team'].str.replace(']', '')

x = 0
for arena in MLBteam_arena['ArenaAPI']:
    target_arena = arena

    target_url = (f'https://maps.googleapis.com/maps/api/geocode/json?'
        'address={0}&key={1}').format(target_arena, gkey)
    geo_data = requests.get(target_url).json()
    lat = geo_data['results'][0]['geometry']['location']['lat']
    lng = geo_data['results'][0]['geometry']['location']['lng']
    address = geo_data['results'][0]['formatted_address']
    arena_ltlg.append({"team" : MLBteam_arena['Team'][x],
                     "location" : [lat, lng],
                       "address" : address,
                     "arena" : MLBteam_arena['Name'][x],
                      "sport" : "MLB"})
    x+=1

# pprint(arena_ltlg)

<ipython-input-4-892679a48f15>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  MLBteam_arena['ArenaAPI'] = [re.sub(r'\s+', '%20', x) for x in MLBteam_arena['Name']]
<ipython-input-4-892679a48f15>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  MLBteam_arena['Team'] = MLBteam_arena['Team'].replace({'nb 2': '', 'nb 3':''}, regex=True)
<ipython-input-4-892679a48f15>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value i

# NBA

In [5]:
NBAurl = 'https://en.wikipedia.org/wiki/List_of_National_Basketball_Association_arenas'
tables = pd.read_html(NBAurl, header=[0])
NBAdf = tables[0]
NBAteam_arena = NBAdf[['Team(s)','Arena']]

NBAteam_arena['ArenaAPI'] = [re.sub(r'\s+', '%20', x) for x in NBAteam_arena['Arena']]
NBAteam_arena['ArenaAPI'] = NBAteam_arena['ArenaAPI'].replace({'&': ''}, regex=True)

x = 0
for arena in NBAteam_arena['ArenaAPI']:
    target_arena = arena

    target_url = (f'https://maps.googleapis.com/maps/api/geocode/json?'
        'address={0}&key={1}').format(target_arena, gkey)
    geo_data = requests.get(target_url).json()
    lat = geo_data['results'][0]['geometry']['location']['lat']
    lng = geo_data['results'][0]['geometry']['location']['lng']
    address = geo_data['results'][0]['formatted_address']
    arena_ltlg.append({"team" : NBAteam_arena['Team(s)'][x],
                     "location" : [lat, lng],
                       "address" : address,
                     "arena" : NBAteam_arena['Arena'][x],
                      "sport" : "NBA"})
    x+=1

# pprint(arena_ltlg)

<ipython-input-5-9f1593f2213c>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  NBAteam_arena['ArenaAPI'] = [re.sub(r'\s+', '%20', x) for x in NBAteam_arena['Arena']]
<ipython-input-5-9f1593f2213c>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  NBAteam_arena['ArenaAPI'] = NBAteam_arena['ArenaAPI'].replace({'&': ''}, regex=True)


# MLS

In [6]:
MLSurl = 'https://en.wikipedia.org/wiki/List_of_Major_League_Soccer_stadiums'
tables = pd.read_html(MLSurl, header=[0])
MLSdf = tables[1]
MLSteam_arena = MLSdf[['Franchise','Stadium']]

MLSteam_arena['ArenaAPI'] = [re.sub(r'\s+', '%20', x) for x in MLSteam_arena['Stadium']]

x = 0
for arena in MLSteam_arena['ArenaAPI']:
    # Fixing Portland Timbers
    if arena == 'Providence%20Park':
        arena = 'Providence%20Park%20Arena'
    target_arena = arena

    target_url = (f'https://maps.googleapis.com/maps/api/geocode/json?'
        'address={0}&key={1}').format(target_arena, gkey)
    geo_data = requests.get(target_url).json()
    lat = geo_data['results'][0]['geometry']['location']['lat']
    lng = geo_data['results'][0]['geometry']['location']['lng']
    address = geo_data['results'][0]['formatted_address']
    arena_ltlg.append({"team" : MLSteam_arena['Franchise'][x],
                     "location" : [lat, lng],
                       "address" : address,
                     "arena" : MLSteam_arena['Stadium'][x],
                      "sport" : "MLS"})
    x+=1

# pprint(arena_ltlg)

<ipython-input-6-33dd64b3feec>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  MLSteam_arena['ArenaAPI'] = [re.sub(r'\s+', '%20', x) for x in MLSteam_arena['Stadium']]


In [7]:
with open('arenaInfo.json', 'w') as f:
    json.dump(arena_ltlg, f)

In [8]:
# stop program run timer
stop = timeit.default_timer()

# view how long it took to run
# try and find improvments to speed up later
print('Time to run:', stop - start, 'seconds')

Time to run: 73.850689424 seconds
